# To Dodge or Not to Dodge?: Predicting League of Legends Match Outcomes
This notebook summarizes a project I worked on with other team members while taking Data Science for Business Analytics at New York University while pursuing my MBA.  My role on the team was data preparation/cleaning, modeling, and performance evaluation.

## Purpose
The purpose of this study is to develop a model to predict the likelihood of a team winning a League of Legends match only using information available prior to the start of a match.  The model would provide a recommendation to players to "dodge" a League of Legends match and avoid the consequences of defeat when the probability of victory is low enough such that the nominal LP loss from dodging the match is more beneficial to the player than attempting to win the match.  This recommendation tool would increase player enjoyment of the game while working to optimize a player's LP gain per unit of time played.

## Background
League of Legends is a strategic action game of the multiplayer online battle arena (“MOBA”) genre developed by Riot Games for PC and Mac. Before each battle, two teams of five players each are assembled by a matchmaking system that groups players of similar skill levels together. The system’s primary determinant of skill level is a hidden calculation similar to the Elo system in chess (and many matchmaking systems in multiplayer games today). They roughly mirror this system with their League Points (LP) system, a publicly visible ranking in which players gain LP each time their team wins a match and lose LP if their team loses. After each match, players gain or lose an amount of LP computed by comparing the existing LP values of their teammates and opponents. For example, players might be rewarded more for victory and penalized less for defeat when facing against a team with a higher aggregate rank (and thus, a lower probability of victory for the players).

Before a match begins, players take turns choosing from over a hundred available characters, or “Champions”, to form a team. Each Champion has its own unique abilities and base statistics that define its strengths, weaknesses, and interesting interactions with other characters. As a result, choosing Champions that complement a team’s selections and negate the other team’s selections is critical to the match’s outcome. After all 10 Champions are selected, players have a brief waiting period before the match actually begins, where they have the opportunity to select additional spells that augment their abilities and provide customizable in-game bonuses. Although unadvertised, this is also an opportunity in which players can exit the lobby to cancel the match and prevent it from starting. The League of Legends community has dubbed this behavior of leaving before the game begins “dodging”. The consequences of dodging are a nominal penalty to the dodging player’s LP (-3 to start, or -10 if repeated within a 16-hour window) in addition to a brief timeout penalty that prevents the player from immediately rejoining the matchmaking queue (6 minutes to start, but increasing to 30 minutes if repeated). Presumably, players would only make this choice if they felt the chance of winning the match was low enough that they would rather leave and accept the penalty of dodging, instead of playing the match out and attempting to win.

## Model Results Summary
Two datasets were evaluated: approximately 10,000 matches from patch 9.6 and 8,000 matches from patch 9.9.  The target variable was a binary classification for the match outcome (win or loss).  Each data set was modeled using logistic regression, a decision tree classifier, and naive Bayes using the match result as the target variable. These models were chosen because they are cheap to run, easy to tune, and can handle a binary classification as a target variable. Initially, k-nearest-neighbor was used as well; however, the computing cost was prohibitive for this evaluation. Each model was trained on 80% of the randomly shuffled data and tested on the remaining 20%.  The reported accuracy for all models is on the test data set.  

The best performing model was logistic regression, and the accuracy of the model for each data set is summarized in the table and ROC curve below. Of these data sets, the data set that combined all of the available data had the greatest accuracy.

| Model | Highly-ranked players (version 9.6) | Highly-ranked players (version 9.9) | Casual players (version 9.9) | Highly-ranked + casual players (version 9.9) | All data across both game versions |
|---|---|---|---|---|---|
|Logistic Regression*   |0.519 ∓ 0.011   |0.507 ∓ 0.013   |0.518 ∓ 0.009   |0.521 ∓ 0.014   |0.529 ∓ 0.006   |

**GridSearch on C with 10 cross-validation folds*

<img src="pictures/ROC_Curve_all_data.png" width="400">

*ROC Curve using logistic regression on all data across both game versions. The optimal threshold occurs at 0.53 where the difference between the FPR and the TPR is 0.54%.*

## Improving the Model Accuracy

Two notable efforts were made to improve the model’s performance. The first was an introduction of new data implemented as a feature. This feature was the win rate of the champion selected by each player. The win rate was mined from a data table issued by Lolalytics.com who analyzes all matches in the current version of the game to generate the win rates for each champion. When this feature was included in the model, there was a slight improvement in accuracy. The logistic regression accuracy was 0.534 ∓ 0.010, an improvement over the previous best result of 0.529 ∓ 0.006.
 
The second effort was to introduce industry knowledge to the model. It is expected that with enough data, a machine learning model would eventually pick up on these patterns, but the limited amount of data was a constraint of this study. There are certain combinations of selections made prior to the match that are strategically poor and greatly inhibit a player’s impact on the match outcome. By introducing rules into the data, a feature column was created to signify if any player on the team chose a poor set of pre-match selections. By introducing this variable, the logistic regression accuracy improved to 0.536 ∓ 0.011, a slight improvement. Only 4.1% of rows in the data had at least 1 player identified as making a poor pre-match selection, which can explain the limited impact the feature had on the model. The ROC curve after introducing the additional features is shown below. When comparing the lower left end of the two ROC curves (before and after adding the additional features), there is a higher TPR for a given FPR, potentially indicating that the added features reduced the false positive rate at high probability thresholds.

<img src="pictures/ROC_Curve_new_features.png" width="400">

*ROC Curve using logistic regression on all data across both game versions after adding champion win rate feature and engineering poor pre-match spell selection identification feature. The optimal threshold occurs at 0.55 where the difference between the FPR and the TPR is 2.77%.*


## Other Improvements to the Model
Potential improvements to the model would be additional data since there is evidence that adding data improves the model's accuracy.  The model results suggested that more data does improve the model accuracy, and the learning curve below confirms this. By looking at the learning curve, one could infer that millions of rows of data would be needed to perform significantly better than random selection. Given the number of features in the dataset (1,563), this is not a surprise. Generally, the more features in a data set, the more examples (rows) a machine learning model will need to recognize patterns.

<img src="pictures/Learning_curve_all_data.png" width="400">

*Learning curve on all data across both game versions.*

<img src="pictures/Learning_curve_new_features.png" width="400">

*Learning curve on all data across both game versions after adding champion win rate feature and engineering poor pre-match spell selection identification feature.*

Another consideration of this project and any model that is built should be that it is assumed that Riot is actively balancing the game which would presumably work against any predictions of match outcomes.  In fact, it's likely that Riot has a data science team performing this type of analysis.

## Expected Value
To generate the expected value for a player who would use this model to decide whether or not to dodge a match, we can apply an expected value framework using the expected LP gain.  The table below displays the confusion matrix for the logistic regression model (with the additional features discusssed previously) at a probability threshold of 50%.

<img src="pictures/confusion_matrix.png" width="400">

*Confusion matrix for model using all data and additional features (50% threshold).*

To calculate an expected value the following was assumed: an LP loss of 20 for losing a match, an LP gain of 20 for winning a match, and an LP loss of 3 for dodging a match as outlined in the expected value table below.

<img src="pictures/expected_value.png" width="400">

*Expected value matrix for model decisions.*

Multiplying the expected value by the confusion matrix yields the table below. Due to the poor model accuracy, our model will increase the LP loss in a match by 1.14 on average.

<img src="pictures/expected_value_framework.png" width="400">

*Expected value for model at 50% threshold.*

Ideally, an optimization of expected value would be performed at various thresholds; however, due to the poor accuracy of our model, there is no point at which the threshold would result in a positive LP gain for a player who used our model to decide to dodge matches. As discussed previously, more data and/or additional features to improve the model’s accuracy would also improve the expected value of using the model.